In [26]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Github/cis

/content/drive/MyDrive/Github


In [ ]:
!git init cis5810

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Github/cis5810/.git/


In [ ]:
cd cis5810

/content/drive/MyDrive/Github/cis5810


On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	CIS5810 Final Project.ipynb

nothing added to commit but untracked files present (use "git add" to track)


[master (root-commit) 27a6258] initial commit
 1 file changed, 1 insertion(+)
 create mode 100644 CIS5810 Final Project.ipynb


Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.67 KiB | 190.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/akalkar15/cis5810.git
 * [new branch]      master -> master
Branch 'master' set up to track remote branch 'master' from 'cis5810'.
